In [21]:
# 라이브러리 호출
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [22]:
# 데이터 전처리
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

In [23]:
# 데이터셋 내려받기 및 전처리 적용
from torchvision.datasets import MNIST

download_root = '../data/MINST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [24]:
# 데이터셋 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
# 일반적으로 검증과 테스트 용도의 데이터셋은 섞어서 사용하지 않지만, 다양한 학습을 위해 True로 지정
valid_loader = DataLoader(valid_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

In [25]:
# 변수 값 선정
batch_size, n_iter = 100, 6000
num_epochs = n_iter / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [36]:
# GRU 셀 네트워크
class GRUcell(nn.Module):

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUcell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
        
    def forward(self, x, hidden):
        x = x.view(-1, x.size(1))

        # LSTM 셀에서는 gates를 x2h + h2h로 정의했지만, GRU에서는 개별적인 상태를 유지
        gate_x = self.x2h(x)
        gate_h = self.h2h(hidden)
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()

        # 총 세 개의 게이트(망각, 입력, 새로운 게이트)를 위해 세 개로 쪼갭니다.
        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_x.chunk(3, 1)

        resetgate = F.sigmoid(i_r + h_r)
        inputgate = F.sigmoid(i_i + h_i)
        # 새로운 게이트는 탄젠트 활성화 함수가 적용된 게이트
        newgate = F.tanh(i_n + (resetgate * h_n))

        hy = newgate + inputgate * (hidden - newgate)
        return hy

In [37]:
# 전반적인 네트워크 구조
class GRUModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        # 앞에서 정의한 GRUcell 함수를 불러옵니다.
        self.gru_cell = GRUcell(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
    
        outs = []
        # LSTM 셀에서는 셀 상태에 대해서도 정의했었지만, GPU 셀에서는 셀은 사용하지 않음
        hn = h0[0, :, :]

        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:, seq, :], hn)
            outs.append(hn)
        out = outs[-1].squeeze()
        out = self.fc(out)
        return out

In [38]:
# 옵티마이저와 손실 함수 설정
input_dim, hidden_dim, layer_dim, output_dim = 28, 128, 1, 10

model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
    model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters() , lr=learning_rate)

In [39]:
# 모델 학습 및 성능 검증
seq_dim, iter = 28, 0
loss_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        if torch.cuda.is_available():
            loss.cuda()
        
        loss.backward()
        optimizer.step()

        loss_list.append(loss.item())
        iter += 1

        if iter % 500 == 0:
            correct, total = 0, 0
            for images, labels in valid_loader:
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1, seq_dim, input_dim))
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)

                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            print(f'Iteration: {iter}. Loss: {loss.item()}. Accuracy: {accuracy}')

Iteration: 500. Loss: 2.191502571105957. Accuracy: 21.0
Iteration: 1000. Loss: 1.5212305784225464. Accuracy: 48.11000061035156
Iteration: 1500. Loss: 1.3396415710449219. Accuracy: 60.68000030517578
Iteration: 2000. Loss: 1.0416069030761719. Accuracy: 65.0999984741211
Iteration: 2500. Loss: 1.0897669792175293. Accuracy: 67.62999725341797
Iteration: 3000. Loss: 0.8338577747344971. Accuracy: 72.66999816894531
Iteration: 3500. Loss: 0.7880936861038208. Accuracy: 74.9000015258789
Iteration: 4000. Loss: 0.8187478184700012. Accuracy: 74.88999938964844
Iteration: 4500. Loss: 0.5930017232894897. Accuracy: 78.31999969482422
Iteration: 5000. Loss: 0.7964613437652588. Accuracy: 78.27999877929688
Iteration: 5500. Loss: 0.6568851470947266. Accuracy: 78.48999786376953
Iteration: 6000. Loss: 0.4407331645488739. Accuracy: 81.01000213623047
Iteration: 6500. Loss: 0.5995995998382568. Accuracy: 81.04000091552734
Iteration: 7000. Loss: 0.7176007032394409. Accuracy: 82.37000274658203
Iteration: 7500. Loss: 

In [48]:
# 테스트 데이터셋을 이용한 모델 예측
def evaluate(model, val_iter):
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter:
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            lables = Variable(labels)

        logit = model(images).to(device)
        loss = F.cross_entropy(logit, labels, reduction='sum')
        _, predicted = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        if torch.cuda.is_available():
            corrects += (predicted.cpu() == labels.cpu()).sum()
        else:
            corrects += (predicted == labels).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [49]:
test_loss, test_acc = evaluate(model, test_loader)
print(f'Test Loss: {test_loss:5.2f} | Test Accuracy: {test_acc:5.2f}')

Test Loss:  0.55 | Test Accuracy:  0.82
